In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components):

    label_ = label

    size = len(data)
    mod = size % 10

    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)
    # print(tam)
    # sys.exit(0)

    data = np.vsplit(data, 10)
    label = u.chunkify(label, 10)
    label_ = label_[0:(size-mod)]
    # print(len(data))
    # print(len(label))

    # percent = int(percent)
    # print(percent)
    # print(epsilon)
    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []

    pred, updt = handshake2.handshake2(data, label_, data[0], label[0], data[1], label[1], n_components, n_features, epsilon, percent)
    pred = np.array(pred, dtype=int)
    # print(label[i])
    pred = pred.flatten()
    predicted = pred
    acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

    # print(acc_percent, f1_percent, mcc_percent)
    # sys.exit(0)

    for i in range(1, len(data)-1):
        # print(i)
        pred, upd = handshake2.handshake2(data, label_, data[i], label[i], data[i+1], label[i+1], n_components, n_features, epsilon, percent)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()

        # print(pred)
        acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

        predicted = np.vstack([predicted, pred])
        # print(predicted)
        # sys.exit(0)

        updt += upd
        acc_percent = np.vstack([acc_percent, acc])
        f1_percent = np.vstack([f1_percent, f1])
        mcc_percent = np.vstack([mcc_percent, mcc])


    # print(acc_percent)

    predicted = predicted.flatten()
    lab = label_[tam:]


    score = accuracy_score(lab, predicted)
    f1 = f1_score(lab, predicted, average = 'macro')
    mcc = matthews_corrcoef(lab, predicted)
    # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
    # print(a,score, f1, mcc)
    return score, f1, mcc


In [3]:
poolsize = 300
clusters = 2
n_components = 2
# episilon = 0.15
step = 10

In [4]:
base = '/home/god/Documentos/tcc/datasets/'
list = ['NOAA.txt', 'elec.txt', 'keystroke.txt']
# list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
# array_p = [30]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [5]:
dic = {}
for key, value in database.items():
    dic[int(key)+5] = {}
    for ep in range(0, len(array_ep)):
        for p in range(0, len(array_p)):

            if key == 2:
                n_components = 4
                step = 5
            adr = value
            dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

            score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_ep[ep], n_components)

            aux = np.hstack([score, f1, mcc])
            result = np.vstack([result, aux])

            a = str(array_ep[ep])
            b = str(array_p[p])
            name = a +' / '+ b
            dic[int(key)+5][name] = aux

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


IndexError: index -1 is out of bounds for axis 1 with size 0

In [ ]:
for p_id, p_info in dic.items():
    print('dataset :', p_id)

    for key in p_info:
        print(key, ':', p_info[key])